# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. <br/>
**Задача:** <br/>   **разобраться влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.**<br/>   Входные данные от банка — статистика о платёжеспособности клиентов. <br/>
**Вопросы:** <br/> 
Зависимость возврата кредита в срок от:<br/>
**наличия детей  
семейного положения  
уровня дохода  
целей кредита**

Результаты исследования будут 
учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
### Описание данных
**children** — количество детей в семье  
**days_employed** — общий трудовой стаж в днях  
**dob_years** — возраст клиента в годах  
**education** — уровень образования клиента  
**education_id** — идентификатор уровня образования  
**family_status** — семейное положение  
**family_status_id** — идентификатор семейного положения  
**gender** — пол клиента  
**income_type** — тип занятости  
**debt** — имел ли задолженность по возврату кредитов  
**total_income** — ежемесячный доход  
**purpose** — цель получения кредита  

### Шаг 1. Прочитаем данные, импортируем библиотеки, изучим общую информацию. 

In [1]:
import pandas as pd
#from pymystem3 import Mystem
from nltk.stem import SnowballStemmer
from collections import Counter
import numpy as np
import math as mth

In [2]:
debtors = pd.read_csv('data.csv')
debtors.head(5) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
debtors.tail(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21506,1,-1556.249906,33,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145541.993320,свадьба
21507,1,-79.832064,32,среднее,1,гражданский брак,1,F,госслужащий,0,98180.279152,сделка с подержанным автомобилем
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля
21511,0,-612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,-165.377752,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,-1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,-280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости


In [4]:
debtors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
debtors.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [6]:
debtors.duplicated().sum()

54

In [7]:
#найдем пропуски
debtors.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [8]:
debtors[debtors['days_employed'].isin(['NaN'])]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [9]:
# смотрим значений по детям?
debtors.sort_values('children')['children'].value_counts().sort_index()

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

In [10]:
debtors['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [11]:
debtors['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [12]:
debtors['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [13]:
debtors['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [14]:
debtors['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [15]:
debtors['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [16]:
#debtors['dob_years'].unique()
debtors['dob_years'][(debtors['dob_years'] < 19) & (debtors['dob_years'] > 0)].count()
debtors['dob_years'][debtors['dob_years'] == 0].count()

101

In [17]:
### Выводы по данным

1. **children**: -1 ребенок (47 значений) - артефакт? можно удалить, заменить на 0 или зачесть в +1. Заменю на 1 - поскольку прилипший ноль в этих данных похоже норма, а количество значений не критично для анализа. 20 детей (76 значений) - многоват. конечно, но это для меня, а так - многодетная семья.
2. **days_employed**: **ПРОПУСКИ**: трудно предположить откуда, информацию о заемщике обычно собирают тщательно, может потерялись при выгрузке. В поставленных в ТЗ задачах не фигурирует и, можно предположить, что влияние на надежность заемщика данный параметр если и оказывает, то косвенноое.Пропуски можно заменить нулями или взять среднее значение по группе занятости. Средее не нашел как  Большое количество **значений с минусом** - можем только предполагать, что минус нарисовался программно или при неправильном заполнении. Будем считат, что это артефакт и **возьмем по модулю**, сообщим о проблеме, если есть кому. есть **выбивающиеся значения**. Можно **поменять тип данных** для красоты и удобства.
3. **dob_years**: нулевые значения в возрасте (101). несовершеннолетних нет. 
3. **education**, **family_status**: регистр скачет
4. **total_income**: ПРОПУСКИ в тех же строках, что и стаж, но связи не вижу, заполнить можно по типу занятости. Интересно сколько пенсионеры получают?! тип данных не нравится. вообще ежемесячный доход важный параметр  - надо посмотреть повнимательней. 
5. **purpose**: цели дублируются - нужно категорировать

Есть дубли - 54 шт.

In [18]:
### Шаг 2. Предобработка данных

In [19]:
#регистр подравняем
debtors['education'] = debtors['education'].str.lower() # регистр в "образовании"
debtors['family_status'] = debtors['family_status'].str.lower() # регистр в "family"
debtors = debtors.loc[debtors['gender'] != 'XNA'] #удалим Хну

In [20]:
# заменим минус детей на просто детей
debtors['children'] = debtors['children'].replace(-1, 1)
debtors['children'].value_counts()

0     14148
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [21]:
#возьмем по модулю стаж
debtors['days_employed'] = debtors['days_employed'].abs()
debtors.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [22]:
#ti_mean = debtors.groupby(['income_type']).mean()
#заменили пропуски ЗП на медиану зарплаты для каждого типа занятости
debtors['total_income'] = debtors.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))


In [23]:
#замеим пропуски стажа
debtors['days_employed'] = debtors.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()))
#debtors.head(20)
debtors.tail(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21506,1,1556.249906,33,высшее,0,гражданский брак,1,F,сотрудник,0,145541.993320,свадьба
21507,1,79.832064,32,среднее,1,гражданский брак,1,F,госслужащий,0,98180.279152,сделка с подержанным автомобилем
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21510,2,2326.499216,28,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,приобретение автомобиля
21511,0,612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,165.377752,26,высшее,0,не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,280.469996,27,неоконченное высшее,2,не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости


In [24]:
debtors['days_employed'] = debtors['days_employed'].astype('int')
debtors['total_income'] = debtors['total_income'].astype('int')
debtors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21524 non-null  int64 
 1   days_employed     21524 non-null  int32 
 2   dob_years         21524 non-null  int64 
 3   education         21524 non-null  object
 4   education_id      21524 non-null  int64 
 5   family_status     21524 non-null  object
 6   family_status_id  21524 non-null  int64 
 7   gender            21524 non-null  object
 8   income_type       21524 non-null  object
 9   debt              21524 non-null  int64 
 10  total_income      21524 non-null  int32 
 11  purpose           21524 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 2.0+ MB


In [25]:
#сгруппирую дебторов по типу занятости и заменю у них 0 возраст на средний возраст для типа занятости. округлю.
debtors['dob_years'] = debtors.groupby('income_type')['dob_years'].transform(lambda x: x.replace(0, x.mean()))

In [26]:
debtors['dob_years'][debtors['dob_years'] == 0].count()

0

In [27]:
debtors = debtors.drop_duplicates().reset_index(drop=True)

In [28]:
debtors.duplicated().sum()

0

In [29]:
## Категоризация данных

In [30]:
### Лематизируем цели кредита, что бы определить их по категориям

In [31]:
list_purpose = debtors['purpose'].unique()
list_purpose

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [32]:
m = Mystem()
list_purpose = debtors['purpose'].unique()
list_lemmas = []
for i in list_purpose:
    lemma = m.lemmatize(i)
    list_lemmas += filter(lambda x: x not in [' ', '\n', 'с', 'со', 'на'], lemma) 
Counter(list_lemmas)

NameError: name 'Mystem' is not defined

In [ ]:
purpose_category = ['жилье', 'недвижимость', 'строительство', 'ремонт', 'свадьба', 'автомобиль', 'образование']
def purpose_lemm(text):
    lemmas = m.lemmatize(text)
    for lemma in lemmas:
        for category in purpose_category:
            if category in lemma:
                return category

In [33]:
debtors['purpose_lemmatize'] = debtors['purpose'].apply(purpose_lemm)
debtors.head()

NameError: name 'purpose_lemm' is not defined

In [ ]:
def categ (row):
    if 'жилье' in row:
        return 'недвижимость'
    if 'строительство' in row:
        return 'недвижимость'
    if 'ремонт' in row:
        return 'недвижимость'
    else:
        return row
debtors['purpose_cat'] = debtors['purpose_lemmatize'].apply(categ)
debtors.head(20)

In [ ]:
debtors['purpose_lemmatize'].value_counts()

In [ ]:
### Категоризируем доходы

In [ ]:
#поделим уровень дохода поримерно по центральному региону на категории
def total_income_cat(row):
    if row['total_income'] <= 50000:
        return 'ниже среднего'
    elif 50000 < row['total_income'] <= 120000:
        return 'средний'
    elif 120000 < row['total_income'] < 500000:
        return 'зажиточный'
    else:
        return 'богатый'
#добавим категории в таблицу
debtors['total_income_cat'] = debtors.apply(total_income_cat, axis = 1) 
#debtors.head(5)

In [ ]:
### Категоризируем количество детей

In [ ]:
# у нас многодетной считается семья с 3 детьми, поэтому сделаем 0 детей, 1-2, и многодетные
def children_category(row):
    if row['children'] == 0:
        return '0 детей'
    elif  1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'
#добавим в таблицу
debtors['children_category'] = debtors.apply(children_category, axis=1) 
#debtors.head()

### Шаг 3. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
child_vs_debt = pd.DataFrame() #создадим новую таблицу
child_vs_debt['child_count'] = debtors.groupby('children')['debt'].count() #количество значений для каждого количества детей
child_vs_debt['child_sum'] = debtors.groupby('children')['debt'].sum() # суммируе наши 0 и 1
child_vs_debt['child_result'] = child_vs_debt['child_sum'] / child_vs_debt['child_count'] #посчитаем отношение 
child_vs_debt.sort_values('children', ascending = False)

In [ ]:
debtors.pivot_table('debt', 'children_category')

Однозначно бездетные показали себя более платежеспособными заемщиками.Прослеживается увеличения числа просроченных задолжностей с ростом количества детей. Заемщики которые имеют 1-2 детей и многодетные имеют небольшой разборс в значениях по задолжностям. Ответ: зависимость есть - наличие детей отрицательно сказывается на платежеспособности заемщика. 

### Есть ли зависимость между семейным статусом и возвратом кредита в срок

In [ ]:
family_status_vs_debt = pd.DataFrame()
family_status_vs_debt['count_fs'] = debtors.groupby('family_status')['debt'].count()
family_status_vs_debt['sum_fs'] = debtors.groupby('family_status')['debt'].sum()
family_status_vs_debt['sum_per_count'] = family_status_vs_debt['sum_fs'] / family_status_vs_debt['count_fs']
family_status_vs_debt.sort_values('sum_per_count', ascending = False)

In [ ]:
debtors['debt'].mean()

**Вывод:** По количеству задолжностей "гражданский брак" стремится к "не женатым"!(9,3 и 9,7).Выходит, что штамп в паспорте все-таки что то значит. Разведенные люди, напротив, даже более ответственны, чем женатые\замужнии.(7,1 против 7,5)

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
#поделим уровень дохода поримерно по центральному региону на категории
#def total_income_cat(row):
    #if row['total_income'] <= 50000:
    #    return 'ниже среднего'
   # elif 50000 < row['total_income'] <= 120000:
  #      return 'средний'
 #   elif 120000 < row['total_income'] < 500000:
 #       return 'зажиточный'
   # else:
      #  return 'богатый'#
#добавим категории в таблицу
#debtors['total_income_cat'] = debtors.apply(total_income_cat, axis = 1) 
#debtors.head(5)
#методом сводных таблиц  установим зависимость возврата кредита от уровня дохода
pd.pivot_table(debtors, index = 'total_income_cat', values = 'debt').sort_values('debt')

### Вывод

Есть зависимость возврата кредита от уровня дохода. Люди с уровнем дохода выше среднего не имеют проблем с возвратом кредита. Кредиторы с уровнем дохода ниже среднего оказались более ответственными по сравнению с кредиторами со средним уровнем дохода

### Как разные цели кредита влияют на его возврат в срок?

In [ ]:
#debtors.head()
pd.pivot_table(debtors, index = 'purpose_lemmatize', values = 'debt').sort_values('debt')

### Вывод

Владельцы жилья и те, кто его преобретает или строит - наиболее ответственные платильшщики. Кредиты на недвижимость можно выдавать смело. Студенты и автовладельцы в конце списка, и если автомобиль можно забрать за долги, то знания монетизировать сложнее. Так что студенты - худший вариант для выдачи кредита.

### Шаг 4. Общий вывод

**Перед нами стояла
задача:** <br/>   **разобраться влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.**
<br/> **Вопросы: 
<br/> Зависимость возврата кредита в срок от:<br/>
наличия детей  
семейного положения  
уровня дохода  
целей кредита**

<br/>   В нашем распоряжении были входные данные от банка — статистика о платёжеспособности клиентов. <br/>
На первом этапе мы проанализировали данные выявили и устранили следующие проблемы:

1. **children**: -1 ребенок (47 значений) - артефакт? можно удалить, заменить на 0 или зачесть в +1. Заменю на 1 - поскольку прилипший ноль в этих данных похоже норма, а количество значений не критично для анализа. 20 детей (76 значений) - многоват. конечно, но это для меня, а так - многодетная семья.
2. **days_employed**: **ПРОПУСКИ**: трудно предположить откуда, информацию о заемщике обычно собирают тщательно, может потерялись при выгрузке. В поставленных в ТЗ задачах не фигурирует и, можно предположить, что влияние на надежность заемщика данный параметр если и оказывает, то косвенноое.Пропуски можно заменить нулями или взять среднее значение по группе занятости. Средее не нашел как  Большое количество **значений с минусом** - можем только предполагать, что минус нарисовался программно или при неправильном заполнении. Будем считат, что это артефакт и **возьмем по модулю**, сообщим о проблеме, если есть кому. есть **выбивающиеся значения**. Можно **поменять тип данных** для красоты и удобства.
3. **dob_years**: нулевые значения в возрасте (101). несовершеннолетних нет. 
3. **education**, **family_status**: регистр скачет
4. **total_income**: ПРОПУСКИ в тех же строках, что и стаж, но связи не вижу, заполнить можно по типу занятости. Интересно сколько пенсионеры получают?! тип данных не нравится. вообще ежемесячный доход важный параметр  - надо посмотреть повнимательней. 
5. **purpose**: цели дублируются - нужно категорировать, дубли - 54 шт.

**<br/> На следующем этапе провели катеноризауию данных:**

<br/> Выделили в группы основные цели кредита
<br/> Создали круппы по количеству детей, заемщиков от 0 до многодетного
<br/> Доходы заемщиков распределили по группам от ниже среднего до богатого

**<br/> После категоризации прступили к анализу данных и ответили на поставленные вопросы. Получив требуемые зависимости, мы можем сделать следующие выводы:**

**Наличие детей и возврат кредита в срок**
<br/> Однозначно бездетные показали себя более платежеспособными заемщиками.Прослеживается увеличения числа просроченных задолжностей с ростом количества детей. Ответ: зависимость есть - наличие детей отрицательно сказывается на платежеспособности заемщика. Думаю заемщики с детьми чаще попадают в просрочку, потому что в приоритет ставят нужды детей и семьи (собрать в школу, полечить, поучить, отправить на отдых). Заемщик без детей, если он в целом ответственный человек, таких обременений не имеет.

**Семейное положение и возврат кредита в срок**
<br/>По количеству задолжностей "гражданский брак" стремится к "не женатым"!(9,3 и 9,7).Выходит, что штамп в паспорте все-таки что то значит. Разведенные люди, напротив, даже более ответственны, чем женатые\замужнии.(7,1 против 7

<br/>Есть зависимость возврата кредита от уровня дохода. Люди с уровнем дохода выше среднего не имеют проблем с возвратом кредита. Кредиторы с уровнем дохода ниже среднего оказались более ответственными по сравнению с кредиторами со средним уровнем дохода

<br/>Владельцы жилья и те, кто его преобретает или строит - наиболее ответственные платильшщики. Кредиты на недвижимость можно выдавать смело. Студенты и автовладельцы в конце списка, и если автомобиль можно забрать за долги, то знания монетизировать сложнее. Так что студенты - худший вариант для выдачи кредита. 

<br/> Люди семейные с высоким уровнем дохода, без детей, берущие кредит на покупку жилья - идеальные заемщики. Из интересных результатов - заемщики с низким уровнем дохода более ответственные, чем остальные (в зависимости от целей и пр. условий, можно кредитовать). Не стоит давать деньги на автомобиль, если не отлажена система отьема авто за долги.  